In [2]:
# Importation des librairies nécessaires
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from soumabkar_films_sdk import MovieClient, MovieConfig
import time
import json
from collections import Counter, defaultdict
from pathlib import Path

d:\training\Udemy\De la Modélisation SQLAlchemy a l API et a la Data App  maitrisez Python grace un projet 100pour100 pratique, réel et complet\projet_code\films-analytics\.venv\Lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [3]:
# Dossiers
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

In [4]:
# Connexion à l'API via le SDK

config = MovieConfig(movie_base_url="https://movie-backend-cm2v.onrender.com")
client = MovieClient(config=config)

# Vérification que l'API est opérationnelle
client.health_check()

MOVIE_API_BASE_URL in MovieConfig init: https://movie-backend-cm2v.onrender.com


{'message': 'API MovieLens opérationnelle'}

In [5]:
# Récupération des stats de l'API
analytics = client.get_analytics()
analytics

AnalyticsResponse(movie_count=9742, rating_count=100836, tag_count=3683, link_count=9742)

In [6]:
# Système sans mise en cache

# Initialisation du compteur de genres
genre_counter = Counter()

# Paramètres pour le batching
limit = 1000
skip = 0

while True:
    batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
    if not batch:
        break
    
    # On extrait les genres du lot et les compte
    for movie in batch:
        genres = movie.get("genres", "")
        genre_list = genres.split("|") if genres else []
        genre_counter.update(genre_list)
    
    skip += limit
    time.sleep(0.5)  # Pour respecter l’API

# Conversion du Counter en DataFrame
genre_df = pd.DataFrame(genre_counter.items(), columns=["genre", "count"])
genre_df = genre_df.sort_values("count", ascending=False).head(10)
genre_df

# Bar chart horizontal
fig = px.bar(
    genre_df,
    x="count",
    y="genre",
    title="Top 10 genres par nombre de films",
    labels={"genre": "Genre", "count": "Nombre de films"},
    color="count",
    color_continuous_scale="viridis",
    orientation='h'  # ← clé pour l'affichage horizontal
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},  # trie du haut vers le bas
    height=500
)

fig.show()

In [8]:
####### Avec Système de mise en cache #########
api_movie_count = analytics.movie_count
print(api_movie_count)

genre_data_file = output_dir / "genre_df.parquet"
meta_file = output_dir / "meta.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    meta = {}
    cached_movie_count = 0

# Décision : utiliser le cache ou recalculer
if genre_data_file.exists() and cached_movie_count == api_movie_count:
    print("Chargement des données depuis le cache...")
    genre_df = pd.read_parquet(genre_data_file)
else:
    print("Mise à jour des données depuis l'API...")
    # Initialisation du compteur de genres
    genre_counter = Counter()

    # Paramètres pour le batching
    limit = 1000
    skip = 0

    while True:
        batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break

        # On extrait les genres du lot et les compte
        for movie in batch:
            genres = movie.get("genres", "")
            genre_list = genres.split("|") if genres else []
            genre_counter.update(genre_list)

        skip += limit
        time.sleep(0.5)  # Pour respecter l’API

    # Conversion du Counter en DataFrame
    genre_df = pd.DataFrame(genre_counter.items(), columns=["genre", "count"])
    genre_df = genre_df.sort_values("count", ascending=False).head(10)

    # Sauvegarde
    genre_df.to_parquet(genre_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count}, f)

# Affichage Plotly
fig = px.bar(
    genre_df,
    x="count",
    y="genre",
    title="Top 10 genres par nombre de films",
    labels={"genre": "Genre", "count": "Nombre de films"},
    color="count",
    color_continuous_scale="viridis",
    orientation='h'
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    height=500
)

fig.show()

9742
Mise à jour des données depuis l'API...


In [9]:
import re

# === Dossiers ===
#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

yearly_data_file = output_dir / "movies_by_year.parquet"
meta_file = output_dir / "meta_movies_by_year.json"

# === Récupération du nombre total de films via analytics ===
#analytics = client.get_analytics()
api_movie_count = analytics.movie_count

# === Lecture du cache s’il existe ===
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    cached_movie_count = 0

# === Utilisation du cache ou recalcul ===
if yearly_data_file.exists() and cached_movie_count == api_movie_count:
    print("Chargement des données depuis le cache...")
    df_yearly = pd.read_parquet(yearly_data_file)

else:
    print("Extraction des années depuis l’API...")

    # === Initialisation ===
    year_counter = Counter()
    skip = 0
    limit = 500
    year_pattern = re.compile(r"\((\d{4})\)$")

    while True:
        batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break

        for movie in batch:
            title = movie.get("title", "")
            match = year_pattern.search(title)
            if match:
                year = int(match.group(1))
                year_counter[year] += 1

        skip += limit
        time.sleep(0.5)

    # === Construction du DataFrame ===
    df_yearly = pd.DataFrame(sorted(year_counter.items()), columns=["year", "movie_count"])

    # === Sauvegarde du cache ===
    df_yearly.to_parquet(yearly_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count}, f)

# === Affichage avec Plotly ===
fig = px.bar(
    df_yearly,
    x="year",
    y="movie_count",
    title="Nombre total de films par année (basé sur le titre)",
    labels={"year": "Année", "movie_count": "Nombre de films"},
)

fig.update_layout(
    xaxis_title="Année",
    yaxis_title="Nombre de films",
    height=500
)

fig.show()

Extraction des années depuis l’API...
